In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset


In [310]:
torch.cuda.is_available()

False

In [76]:
# df = pd.read_csv("../NER_Dataset/ner-gmb/train.txt", sep = " ", header = None, names = ['token', 'postag', 'word', 'nertag'], skip_blank_lines=False)
# df_list = np.split(df, df[df.isnull().all(1)].index) 
# #add sentence number to each dataframe and then combine. Also for each dataframe remove the first row

# def add_sentence_number(df, i):
#     '''adds a column to df in which all rows have value i'''
#     df['sentence'] = i
#     return df

# new_dflist = []
# for i, df in enumerate(df_list[1:]):
#     df.reset_index(drop=True, inplace = True)
#     df.dropna(inplace = True)
#     df = add_sentence_number(df, i)
#     new_dflist.append(df)

# len(new_dflist)

# def combine_dataframes(df_list):

#     return pd.concat(df_list)


# df = combine_dataframes(new_dflist)
# df.reset_index(drop=True, inplace = True)

In [118]:
# reading text file in python and making list of sentences (list of lists) and list of tags(list of lists)

traindatapath = "../NER_Dataset/ner-gmb/train.txt"
# train_sent = []
# train_tags = []
all_words = []
all_tags = []
with open(traindatapath) as f:
    lines = f.readlines()
    sent_num = 0
    for line in lines[1:]: #1: so that the first blank line isn't taken into account
        if(line == "\n"):
            sent_num+=1
            train_sent.append(sentence)
            train_tags.append(tags)
            sentence = []
            tags = []
        else:
            line_sep = line.split(sep = " ")
            all_words.append(line_sep[0])
            all_tags.append(line_sep[3][:-1])
            
words = list(set(all_words))
tags = list(set(all_tags))

vocab = {}
vocab['<pad>'] = 0 # for padding input sequences
for i, word in enumerate(words):
    vocab[word] = i+1
    
nertags = {}
nertags['padtag'] = 0
for i,nertag in enumerate(tags):
    nertags[nertag] = i+1

In [143]:
train_sent = []
train_tags = []
with open(traindatapath) as f:
    lines = f.readlines()
    sent_num = 0
    sentence = []
    tags = []
    for line in lines[1:]: #1: so that the first blank line isn't taken into account
        if(line == "\n"):
            sent_num+=1
            train_sent.append(sentence)
            train_tags.append(tags)
            sentence = []
            tags = []
        else:
            line_sep = line.split(sep = " ")
            sentence.append(vocab[line_sep[0]])
            tags.append(nertags[line_sep[3][:-1]])

In [147]:
print(train_sent[:2])
print(train_tags[:2])
print(len(words), "unique words")
print("{} unique tags".format(len(tags)))

[[4005, 19151, 1187, 3403, 20949, 18982, 11604, 4005, 10587, 17286, 22237, 1010, 9697, 22237, 22227, 17286, 752, 12471, 18281, 6116, 18268, 23439, 16756, 659, 14017, 15937, 13748, 4005, 6014, 22244, 20949, 13748, 12471, 3962, 23439, 2606, 7225, 21870, 20168, 7997, 13403], [14355, 20949, 4005, 699, 12471, 16866, 32, 21932, 6370, 9270, 21870, 7362, 21328, 9878, 14066, 4005, 10281, 23439, 4005, 16677, 8864, 13748, 6200, 13403]]
[[16, 9, 16, 16, 16, 4, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 6, 16, 16, 16, 16], [16, 16, 16, 16, 16, 16, 16, 16, 16, 6, 16, 6, 16, 16, 16, 16, 16, 16, 16, 13, 16, 16, 9, 16]]
24136 unique words
0 unique tags


In [250]:
# padding the sentences at the end
seq_maxlen = max(len(x) for x in train_sent)
x_lengths = [len(x) for x in train_sent]
Xtrain = []
Ytrain = []
for sent, tags in zip(train_sent, train_tags):
    length_toappend = seq_maxlen - len(sent)
    Xtrain.append(sent+[0]*length_toappend)
    Ytrain.append(tags+[0]*length_toappend)

In [251]:
print(Xtrain[0])
print(Ytrain[0])
print(x_lengths[0])

[4005, 19151, 1187, 3403, 20949, 18982, 11604, 4005, 10587, 17286, 22237, 1010, 9697, 22237, 22227, 17286, 752, 12471, 18281, 6116, 18268, 23439, 16756, 659, 14017, 15937, 13748, 4005, 6014, 22244, 20949, 13748, 12471, 3962, 23439, 2606, 7225, 21870, 20168, 7997, 13403, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[16, 9, 16, 16, 16, 4, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 6, 16, 16, 16, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
41


In [252]:
Xtrain = torch.Tensor(Xtrain)
Ytrain = torch.Tensor(Ytrain)
x_lengths = torch.Tensor(x_lengths)
print(Xtrain.shape, Ytrain.shape, x_lengths.shape)

torch.Size([37206, 104]) torch.Size([37206, 104]) torch.Size([37206])


### Training Data
### using dataloader to make data batches

In [247]:
# def return_lengths(batch):
#     print(batch)
#     (Xbatch, Ybatch) = zip(*batch)
#     xlength = [len(x) for x in Xbatch]
#     return Xbatch, Ybatch, xlength

In [254]:
dataset = TensorDataset(Xtrain, Ytrain, x_lengths)
dataloader = DataLoader(dataset, batch_size= 128)

In [256]:
print(len(next(iter(dataloader))))
type(next(iter(dataloader)))#shape is (batchsize, seqlen)
len(next(iter(dataloader)))

# Xbatch, Ybatch, xlength = next(iter(dataloader))

3


3

In [257]:
print(Xbatch[2].shape)

torch.Size([104])


In [258]:
# # random initialization of word embeddings
# # ie define tensor of size (number of words, embedding dimesnion)
# embed_dim = 100
# num_words = len(vocab)
# random_embed = torch.rand(num_words, embed_dim)
# print(random_embed.shape)
# print(random_embed[1])

# BiLSTM Model

In [305]:
class BiLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, total_words, num_class, pretrained = False, pretrained_embed = None):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        
        self.wordembed = nn.Embedding(total_words, embedding_size) #weights initiallized from std normal dist, randomly
        if(pretrained == True):
            self.wordembed.weight = nn.Parameter(pretrained_embed) #pretrained embeds have size (total_words, embedding_size)

        self.bilstm = nn.LSTM(embedding_size,hidden_size, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(2*hidden_size, num_class) # 2 because forward and backward concatenate

    def forward(self, x, xlengths):
        out = self.wordembed(x) # x is of size(batchsize, seq_len), out is of size (batchsize, seq_len, embedding_size = 100)
        
#         out = pack_padded_sequence(out, xlengths, batch_first=True, enforce_sorted=False)
        
        out, (h,c) = self.bilstm(out) #'out' has dimension(batchsize, seq_len, hidden_size)
        
#         out, out_lengths = pad_packed_sequence(out, batch_first=True)
#         assert(out.shape[-1] == self.hidden_size)
#         print(out.shape)

        out = self.linear(out) #now 'out' has dimension(batchsize, seq_len, num_class)

        out = F.softmax(out, dim=2) # take the softmax across the dimension num_class, 'out' has dimension(batchsize, seq_len, num_class)
        
        return out
        
        

In [306]:
model = BiLSTM(embedding_size = 100, hidden_size = 100, total_words = len(vocab), num_class = 18, pretrained = False)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01) 
lossfunction = nn.CrossEntropyLoss()

In [307]:
print(model)

BiLSTM(
  (wordembed): Embedding(24137, 100)
  (bilstm): LSTM(100, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=18, bias=True)
)


In [308]:
# Model is ready now we have to train using cross entropy loss
num_epochs = 100
batch_size = 128
total_batches = len(train_sent)//128 +1
trainloss = []
# validloss = []
for epoch in range(num_epochs):
    for step, (Xbatch, Ybatch, xbatch_len) in enumerate(dataloader):
        #make gradients 0
        optimizer.zero_grad()
        
        #get output from model and claculate loss
        ypred = model(Xbatch.long(), xbatch_len).permute(0, 2, 1)
#         print(ypred.shape, Ybatch.shape)
        
        loss = lossfunction(ypred, Ybatch.type(torch.LongTensor)) #Ybatch has dimension (batchsize, seqlen), ypred has dimension(batchsize, num_classes, seqlen)
        trainloss.append(loss)
        
        #backward and step
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5) # clip gradient to 5
        optimizer.step()
    print(trainloss[-1])
    print(epoch)
        
        
        
        

KeyboardInterrupt: 

In [309]:
print(trainloss)

[tensor(2.8931, grad_fn=<NllLoss2DBackward>), tensor(2.8929, grad_fn=<NllLoss2DBackward>), tensor(2.8923, grad_fn=<NllLoss2DBackward>), tensor(2.8920, grad_fn=<NllLoss2DBackward>), tensor(2.8919, grad_fn=<NllLoss2DBackward>), tensor(2.8914, grad_fn=<NllLoss2DBackward>), tensor(2.8911, grad_fn=<NllLoss2DBackward>), tensor(2.8905, grad_fn=<NllLoss2DBackward>), tensor(2.8902, grad_fn=<NllLoss2DBackward>), tensor(2.8902, grad_fn=<NllLoss2DBackward>), tensor(2.8899, grad_fn=<NllLoss2DBackward>), tensor(2.8892, grad_fn=<NllLoss2DBackward>), tensor(2.8891, grad_fn=<NllLoss2DBackward>), tensor(2.8887, grad_fn=<NllLoss2DBackward>), tensor(2.8883, grad_fn=<NllLoss2DBackward>), tensor(2.8880, grad_fn=<NllLoss2DBackward>), tensor(2.8874, grad_fn=<NllLoss2DBackward>), tensor(2.8871, grad_fn=<NllLoss2DBackward>), tensor(2.8870, grad_fn=<NllLoss2DBackward>), tensor(2.8867, grad_fn=<NllLoss2DBackward>), tensor(2.8860, grad_fn=<NllLoss2DBackward>), tensor(2.8854, grad_fn=<NllLoss2DBackward>), tensor(2.